作業2：使用 TensorFlow 實作 CNN 捕捉 K 線趨勢，並預測股價漲跌（分類任務）。
說明：輸入一張 N 天的 K 線圖判斷未來價格可能漲或跌，label 方式可以使用技術指標輔助，例如：5日均線高於10日均線標註「漲」，5日均線低於10日均線標註「跌」。

In [18]:
import plotly.graph_objects as go
import pandas as pd

# 假設 df 是包含股價資料的 DataFrame
df = pd.read_csv('./stock_data_1301_with_features_0.3years.csv')
df['日期'] = pd.to_datetime(df['日期'])
# 生成 K 線圖 用 plotly.go
fig = go.Figure(data=[go.Candlestick(x=df['日期'],
                                        open=df['開盤價'],
                                        high=df['最高價'],
                                        low=df['最低價'],
                                        close=df['收盤價'],
                                        increasing_line_color='red',
                                        decreasing_line_color='green')])
fig.update_layout(title='K Line Chart', xaxis_title='Date', yaxis_title='Close Price',width=1000,height=600)
fig.show()


In [19]:
import tensorflow as tf

image_height = 865
image_width = 353

# 定義 CNN 模型, using an `Input(shape)` object as the first layer in the model 
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(image_height, image_width, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # 二元分類，使用 sigmoid 激活函數
])

# 編譯模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [20]:
from PIL import Image
import numpy as np

# 讀取圖像
train_image = Image.open('./source/k_line_chart_cut.png')

# 轉換為 NumPy 數組並正規化
img_array = np.array(train_image) / 255.0
if img_array.shape[-1] == 4:
    img_array = img_array[..., :3]
img_array = np.expand_dims(img_array, axis=0)

# 生成標籤
df['Label'] = df['SMA5'] > df['SMA10']

# 獲取標籤
train_labels = df['Label'].values


In [21]:
# model.fit(img_array, train_labels, epochs=10, batch_size=32)